In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time

In [24]:
def parse_xml_tei(file_path):
    print(file_path)
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'xml')
    
    posts = soup.find_all('post')
    results = []
    for post in posts:
        xml_id = post['xml:id']
        try : 
            n = post['n']
        except:
            n = ''
        who = post['who']
        date_attr = [attr for attr in post.attrs if re.match(r'when.*', attr)]
        if date_attr:
            date = post[date_attr[0]]
        else:
            date = None
        texte = ' '.join([x.get_text() for x in post.find_all('p')])
        texte_mod =re.sub("\\\'", "\'", texte)
        texte = re.sub(r'\n\s+', ' ', texte_mod)
        sujet = file_path.replace('F:\Corpus\wikiconflict\cmr-wikiconflits-','').replace('_discu-tei-v1.xml','')
        results.append({'xml_id': xml_id, 'date': date, 'n':n, 'who':who, 'texte':texte, 'sujet':sujet})

    return results

In [25]:
def get_files_in_directory(directory):
    file_list = []
    # Vérifier si le dossier existe
    if os.path.exists(directory):
        # Parcourir tous les fichiers du dossier
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                file_list.append(file_path)
    
    return file_list

In [26]:
def dict_to_dataframe(dictionary):
    df = pd.DataFrame(dictionary)
    return df

In [27]:
path = get_files_in_directory('F:\Corpus\wikiconflict')

In [28]:
posts = [parse_xml_tei(file_path) for file_path in path]

F:\Corpus\wikiconflict\cmr-wikiconflits-bogdanoff_discu-tei-v1.xml
F:\Corpus\wikiconflict\cmr-wikiconflits-chiropratique_discu-tei-v1.xml
F:\Corpus\wikiconflict\cmr-wikiconflits-eolienne_discu-tei-v1.xml
F:\Corpus\wikiconflict\cmr-wikiconflits-histologique_discu-tei-v1.xml
F:\Corpus\wikiconflict\cmr-wikiconflits-ogm_discu-tei-v1.xml
F:\Corpus\wikiconflict\cmr-wikiconflits-psychanalyse_discu-tei-v1.xml
F:\Corpus\wikiconflict\cmr-wikiconflits-qi_discu-tei-v1.xml


In [17]:
path

['F:\\Corpus\\wikiconflict\\cmr-wikiconflits-bogdanoff_discu-tei-v1.xml',
 'F:\\Corpus\\wikiconflict\\cmr-wikiconflits-chiropratique_discu-tei-v1.xml',
 'F:\\Corpus\\wikiconflict\\cmr-wikiconflits-eolienne_discu-tei-v1.xml',
 'F:\\Corpus\\wikiconflict\\cmr-wikiconflits-histologique_discu-tei-v1.xml',
 'F:\\Corpus\\wikiconflict\\cmr-wikiconflits-ogm_discu-tei-v1.xml',
 'F:\\Corpus\\wikiconflict\\cmr-wikiconflits-psychanalyse_discu-tei-v1.xml',
 'F:\\Corpus\\wikiconflict\\cmr-wikiconflits-qi_discu-tei-v1.xml']

In [29]:
all_ = pd.concat([dict_to_dataframe(x) for x in posts])

In [31]:
all_.to_csv('F:\Corpus\\finaux\wikiconflict.csv', sep='\t', encoding='utf-8')

In [85]:
# Exemple d'utilisation
file_path = 'F:\Corpus\wikiconflict\cmr-wikiconflits-bogdanoff_discu-tei-v1.xml'
posts = parse_xml_tei(file_path)

In [ ]:
all_ = pd.concat([dict_to_dataframe(x) for x in posts])

In [2]:
import xml.etree.ElementTree as ET

def parse_reddit_xml(file_path):
    #tree = ET.parse(file_path, parser = ET.XMLParser(encoding = 'utf-8'))
    root =  ET.fromstring(file_path)

    data = []

    for s in root.findall('s'):
        link_id = s.get('link_id')
        subreddit_id = s.get('subreddit_id')
        
        for utt in s.findall('utt'):
            uid = utt.get('uid')
            comment_id = utt.get('comment_id')
            parent_id = utt.get('parent_id')
            score = utt.get('score')
            create_utc = utt.get('create_utc')
            text = utt.text.strip()
            
            data.append({
                'link_id': link_id,
                'subreddit_id': subreddit_id,
                'uid': uid,
                'comment_id': comment_id,
                'parent_id': parent_id,
                'score': score,
                'create_utc': create_utc,
                'text': text
            })
    time.sleep(3)
    return data


In [3]:
def remove_control_characters(xml_file):
    with open(xml_file, 'r', encoding='utf-8') as file:
        xml_content = file.read()

    # Supprimer les caractères de contrôle
    cleaned_content = re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]', '', xml_content)

    return cleaned_content


In [7]:
def clean_string(text):
    # Supprimer les sauts de ligne en trop
    text = re.sub(r'\n+', '\n', text)
    
    # Supprimer les espaces en trop
    text = re.sub(r' +', ' ', text)
    
    # Supprimer les caractères d'échappement '\' suivis de "'"
    text = re.sub(r'\\\'', "'", text)
    
    return text

In [4]:
clean = remove_control_characters(r"F:\Corpus\reddit\archive\final_SPF_2.xml")

In [5]:
d = parse_reddit_xml(clean)

In [12]:
def clean_text_in_dict_list(dict_list):
    cleaned_list = [
        {key: clean_string(value) if key == 'text' else value for key, value in data.items()}
        for data in dict_list
    ]
    return cleaned_list

In [13]:
cleaned_data_list = clean_text_in_dict_list(d)

In [14]:
cleaned_data_list

[{'link_id': '8r1kz',
  'subreddit_id': '2qhjz',
  'uid': '1688932',
  'comment_id': 'c0a62uj',
  'parent_id': '8r1kz',
  'score': '3',
  'create_utc': '1244576002',
  'text': "Ironie : l'article disant qu'on est plus capable de se concentrer plus de 2 pages, en fait 6 !\nJ'ai pas pu aller plus loin que la page 3, même si c'était intéressant !"},
 {'link_id': '8r1kz',
  'subreddit_id': '2qhjz',
  'uid': '786883',
  'comment_id': 'c0a6lmb',
  'parent_id': 'c0a62uj',
  'score': '1',
  'create_utc': '1244621120',
  'text': "Moi-même, j'ai dû me forcer pour arriver jusqu'à la fin. Et pourtant ça ne devient pas moins intéressant. Ca fout les jetons... :)"},
 {'link_id': '8sncs',
  'subreddit_id': '2qhjz',
  'uid': '390497',
  'comment_id': 'c0aawpk',
  'parent_id': '8sncs',
  'score': '1',
  'create_utc': '1245076061',
  'text': 'Service qui sera rendu au contribuable pour la modique some de 70 Millions par an.'},
 {'link_id': '8sncs',
  'subreddit_id': '2qhjz',
  'uid': '32884',
  'comment

In [15]:
df = pd.DataFrame(cleaned_data_list)

In [16]:
df

,link_id,subreddit_id,uid,comment_id,parent_id,score,create_utc,text
0,8r1kz,2qhjz,1688932,c0a62uj,8r1kz,3,1244576002,Ironie : l'article disant qu'on est plus capab...
1,8r1kz,2qhjz,786883,c0a6lmb,c0a62uj,1,1244621120,"Moi-même, j'ai dû me forcer pour arriver jusqu..."
2,8sncs,2qhjz,390497,c0aawpk,8sncs,1,1245076061,Service qui sera rendu au contribuable pour la...
3,8sncs,2qhjz,32884,c0aaxba,c0aawpk,3,1245077396,Eeeeh oui ! 70 millions pour une loi qui aura ...
4,8v13c,2qhjz,796919,c0aj3ov,8v13c,2,1245830384,Est-ce qu'elle a vraiment commis des crimes qu...
...,...,...,...,...,...,...,...,...
1583078,6x2rqg,2qhjz,1933944,dmdl33g,dmdk7qp,2,1504189104,Bah c'est cool pour toi. Moi j'ai jamais vécu ...
1583079,6x2rqg,2qhjz,1577194,dmdlji3,dmdl33g,1,1504189587,Effectivement. On ne peut pas généraliser le t...
1583080,6x2rqg,2qhjz,298487,dmdm1xa,dmdlji3,2,1504190145,Sauf que moi je n'ai pas besoin de généraliser...
1583081,6x2rqg,2qhjz,1581349,dmdm49b,dmdm1xa,0,1504190216,Ca s'appelle un contre exemple... à ton post i...


In [18]:
df.to_csv(r'F:\Corpus\finaux\reddit.csv', sep ='\t', encoding='utf8')